Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [5]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[64:100])
print(valid_size, valid_text[64:100])

99999000 sociation of autonomous individuals 
1000 y working class radicals including t


In [7]:
valid_text

' anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state the word anarchy as most anarchists use it does not imply chaos nihilism or anomie but rather a harmonious anti authoritarian society in place of what are regarded as authoritarian political structures and coercive economic institut

Utility functions to map characters to vocabulary IDs and back.

In [9]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0), id2char(9))

Unexpected character: ï
1 26 0 0
a z   i


Function to generate a training batch for the LSTM model.

In [24]:
batch_size = 8
num_unrollings = 12

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print('train 1:', batches2string(train_batches.next()))
print('train 2:', batches2string(train_batches.next()))
print('valid 1:', batches2string(valid_batches.next()))
print('valid 2:', batches2string(valid_batches.next()))

train 1: ['ons anarchist', 'n from the na', 'gnificant tha', ' drugs confus', 'of the origin', 'at least not ', 'st daily coll', 'cky ricardo t']
train 2: ['ts advocate s', 'ational media', 'an in jersey ', 'sion inabilit', 'nal document ', ' parliament s', 'lege newspape', 'this classic ']
valid 1: [' a']
valid 2: ['an']


In [115]:
train_batches.next()

[array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.

In [25]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [26]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [27]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298413 learning rate: 10.000000
Minibatch perplexity: 27.07
wozirfs n akahlxtctznbxsrefaiimsqa mwolzdnnexgnd  ru fqeabu ushspdeykiliqlne sxu
lim ebavcr q dabivtzkalixvacrpxyp grsjqeet rl isloyryneuthdausgqvoaaqqjltso lmbz
jma ntali oourz dougimvafcaifjucvsgeied nfnycygt pbrur klvx kn kirrif osd no  nh
gdmre crobrr   t llu omrajrcxnmipujuokly eivhlmxsnfaln ei or tfaeytnegdannbqv  d
asv eknetrtselnfy eqa spaotptusmz  phzt wrwdltcamoixccohalab revkpsgkkjjocovs zp
Validation set perplexity: 20.45
Average loss at step 100: 2.705587 learning rate: 10.000000
Minibatch perplexity: 12.16
Validation set perplexity: 13.79
Average loss at step 200: 2.400011 learning rate: 10.000000
Minibatch perplexity: 12.36
Validation set perplexity: 10.09
Average loss at step 300: 2.242891 learning rate: 10.000000
Minibatch perplexity: 9.63
Validation set perplexity: 11.74
Average loss at step 400: 2.014155 learning rate: 10.000000
Minibatch perplexity: 7.75
Validation set 

Validation set perplexity: 5.71
Average loss at step 4500: 1.775104 learning rate: 10.000000
Minibatch perplexity: 5.70
Validation set perplexity: 5.77
Average loss at step 4600: 1.691801 learning rate: 10.000000
Minibatch perplexity: 5.38
Validation set perplexity: 5.94
Average loss at step 4700: 1.703981 learning rate: 10.000000
Minibatch perplexity: 4.58
Validation set perplexity: 5.72
Average loss at step 4800: 1.731012 learning rate: 10.000000
Minibatch perplexity: 4.36
Validation set perplexity: 5.92
Average loss at step 4900: 1.765953 learning rate: 10.000000
Minibatch perplexity: 4.67
Validation set perplexity: 5.67
Average loss at step 5000: 1.630889 learning rate: 1.000000
Minibatch perplexity: 5.57
mowsish austbther the featent are the munnition football fies themected by persi
four be for the conternational films the arction pares which wells the maxal of 
kerely vocal us the tuctsedpam but the guiv resiver iversective k teachlinbus it
y naw the archill of norrearince and m

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [44]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))

  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  wx = tf.concat([ix,fx,cx,ox], axis=1)  # <---- added this
  wm = tf.concat([im,fm,cm,om], axis=1)  # <---- added this
  wb = tf.concat([ib,fb,cb,ob], axis=1)  # <---- added this

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    x = tf.matmul(i, wx) + tf.matmul(o, wm) + wb                       # <---- added this
    xi, xf, xc, xo = tf.split(value=x, num_or_size_splits=4, axis=1)   # <---- added this
    input_gate = tf.sigmoid(xi)
    forget_gate = tf.sigmoid(xf)
    update = xc
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(xo)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [46]:
num_steps = 10001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295797 learning rate: 10.000000
Minibatch perplexity: 27.00
qai ojftorio dbsasihbrfgpttsbrmnlds fploaniejbioztluf ogb  vueiblpjgog nhrkfkhox
bont uyz vsbam atjy ynjlesiaygodkj ckgegilagkyigznqz ayo ih tnttvanw mjevieiukas
i aentbffh x  mlhkyskwxiiqfnujatd lghfofusifnesi rt docertgfwit ltitizcidsitwo  
cbfui a etexiigbl ypgiilonqa gmanwsac jhjlqvrfjzjdni lilwnxmrnslsneuomjxlomag uq
suognehvsveg ogbd lzv ukbaqmdgtgcoef  s  atisk ntnunlfi  igkujuulqn nruapjqtbnfm
Validation set perplexity: 20.59
Average loss at step 100: 2.696001 learning rate: 10.000000
Minibatch perplexity: 11.94
Validation set perplexity: 12.00
Average loss at step 200: 2.367294 learning rate: 10.000000
Minibatch perplexity: 10.47
Validation set perplexity: 10.38
Average loss at step 300: 2.233170 learning rate: 10.000000
Minibatch perplexity: 8.27
Validation set perplexity: 9.52
Average loss at step 400: 2.179252 learning rate: 10.000000
Minibatch perplexity: 8.51
Validation set p

Validation set perplexity: 5.89
Average loss at step 4500: 1.693356 learning rate: 10.000000
Minibatch perplexity: 6.66
Validation set perplexity: 5.74
Average loss at step 4600: 1.732837 learning rate: 10.000000
Minibatch perplexity: 5.40
Validation set perplexity: 6.16
Average loss at step 4700: 1.724926 learning rate: 10.000000
Minibatch perplexity: 6.59
Validation set perplexity: 5.97
Average loss at step 4800: 1.713375 learning rate: 10.000000
Minibatch perplexity: 5.05
Validation set perplexity: 6.31
Average loss at step 4900: 1.709369 learning rate: 10.000000
Minibatch perplexity: 5.61
Validation set perplexity: 5.70
Average loss at step 5000: 1.689141 learning rate: 1.000000
Minibatch perplexity: 5.98
gre book in the bied himptent when satic agen were astens arease or btiter nord 
arland ten liclinedtobotes in throve fease them costern bih mate ray the entingb
ces controctial datey view prokaten hominiant th met the from gocreas an pet the
x such and mates trout in english the 

Validation set perplexity: 4.82
Average loss at step 9100: 1.644340 learning rate: 1.000000
Minibatch perplexity: 6.11
Validation set perplexity: 4.84
Average loss at step 9200: 1.646034 learning rate: 1.000000
Minibatch perplexity: 4.18
Validation set perplexity: 4.82
Average loss at step 9300: 1.610363 learning rate: 1.000000
Minibatch perplexity: 4.20
Validation set perplexity: 4.79
Average loss at step 9400: 1.715570 learning rate: 1.000000
Minibatch perplexity: 6.12
Validation set perplexity: 4.83
Average loss at step 9500: 1.663339 learning rate: 1.000000
Minibatch perplexity: 5.76
Validation set perplexity: 4.77
Average loss at step 9600: 1.581935 learning rate: 1.000000
Minibatch perplexity: 4.29
Validation set perplexity: 4.81
Average loss at step 9700: 1.619453 learning rate: 1.000000
Minibatch perplexity: 7.76
Validation set perplexity: 4.81
Average loss at step 9800: 1.638151 learning rate: 1.000000
Minibatch perplexity: 5.60
Validation set perplexity: 4.83
Average loss at 

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

a) Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

In [53]:
embedding_size = 128 # Dimension of the embedding vector. # <--- This line

num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Emebeding: 
  vocab_embedding = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], -0.1, 0.1)) # <--- This line
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    embedding = tf.nn.embedding_lookup(vocab_embedding, tf.argmax(i, dimension=1)) # <--- This line
    output, state = lstm_cell(embedding, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  sample_input_embedding = tf.nn.embedding_lookup(vocab_embedding, tf.argmax(sample_input, dimension=1))
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(sample_input_embedding, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [54]:
num_steps = 10001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.356768 learning rate: 10.000000
Minibatch perplexity: 28.70
ueuebeseseseseeezelelebehefeeexefewereoevedeiere e ezepeyetewevemelegeueu zeleee
geee ejeleieuevege ele eqefereeeueaeaesewejevezeneeeyeeebeqeeeeejemeneeekesede e
reeeieaebee selerezeseieeesepeseqeserece exeweqebeleeeo pekeieseeele ecez  eeexe
oe efeaebe eeehegeheeelevemeqeieceieaeleteleceiebegejeaekefeie euese efeeejezexe
mehele eyenekeeare eeefewezeaebelede eo  ehedededeeelepexexemeiegewewemeyekeaede
Validation set perplexity: 579.33
Average loss at step 100: 2.860229 learning rate: 10.000000
Minibatch perplexity: 13.41
Validation set perplexity: 16.32
Average loss at step 200: 2.390206 learning rate: 10.000000
Minibatch perplexity: 10.57
Validation set perplexity: 10.81
Average loss at step 300: 2.268013 learning rate: 10.000000
Minibatch perplexity: 14.04
Validation set perplexity: 9.97
Average loss at step 400: 2.246893 learning rate: 10.000000
Minibatch perplexity: 11.73
Validation se

Validation set perplexity: 7.90
Average loss at step 4500: 1.839464 learning rate: 10.000000
Minibatch perplexity: 7.24
Validation set perplexity: 7.34
Average loss at step 4600: 1.786330 learning rate: 10.000000
Minibatch perplexity: 5.70
Validation set perplexity: 7.14
Average loss at step 4700: 1.840564 learning rate: 10.000000
Minibatch perplexity: 6.31
Validation set perplexity: 7.33
Average loss at step 4800: 1.882769 learning rate: 10.000000
Minibatch perplexity: 6.66
Validation set perplexity: 7.63
Average loss at step 4900: 1.835772 learning rate: 10.000000
Minibatch perplexity: 6.92
Validation set perplexity: 7.63
Average loss at step 5000: 1.851455 learning rate: 1.000000
Minibatch perplexity: 6.36
clitic is one nine nine one eight one chortion tradionally danchelly a pacted am
quel that hicationally the a tethadoaft open eration anothe hatatiogh he five th
ur they appeare have but howmen virus of i for the prodets ptho hose controus fo
xing one rechoturd one nine dition s o

Validation set perplexity: 5.70
Average loss at step 9100: 1.693964 learning rate: 1.000000
Minibatch perplexity: 5.08
Validation set perplexity: 5.61
Average loss at step 9200: 1.723659 learning rate: 1.000000
Minibatch perplexity: 5.48
Validation set perplexity: 5.58
Average loss at step 9300: 1.678916 learning rate: 1.000000
Minibatch perplexity: 4.88
Validation set perplexity: 5.66
Average loss at step 9400: 1.662441 learning rate: 1.000000
Minibatch perplexity: 5.78
Validation set perplexity: 5.52
Average loss at step 9500: 1.674979 learning rate: 1.000000
Minibatch perplexity: 5.91
Validation set perplexity: 5.72
Average loss at step 9600: 1.622947 learning rate: 1.000000
Minibatch perplexity: 4.26
Validation set perplexity: 5.65
Average loss at step 9700: 1.748362 learning rate: 1.000000
Minibatch perplexity: 5.25
Validation set perplexity: 5.67
Average loss at step 9800: 1.669339 learning rate: 1.000000
Minibatch perplexity: 6.21
Validation set perplexity: 5.63
Average loss at 

In [55]:
# b) 

In [95]:
bi_grams_set = set(train_text[i:i+2] for i in range(0, len(train_text), 2))

In [114]:
bi_grams_list = sorted(list(bi_grams_set))
bi_grams_vocabulary_size = len(bi_grams_list)


def bi_gram2id(bi_grams_list, bi_gram):
    if bi_gram in bi_grams_list:
      return bi_grams_list.index(bi_gram)+1
    else:
        return 0 
  
def id2bi_gram(bi_grams_list, dictid):
  if dictid > 0:
    return bi_grams_list[dictid]
  else:
    return '  '

print(bi_gram2id(bi_grams_list,' a'), bi_gram2id(bi_grams_list,'sS'), bi_gram2id(bi_grams_list,'az'), bi_gram2id(bi_grams_list,'g '))
print(id2bi_gram(bi_grams_list,0), id2bi_gram(bi_grams_list, 96), id2bi_gram(bi_grams_list,247), id2bi_gram(bi_grams_list, 654))

1 0 53 189
   cp ie xg


In [119]:
batch_size = 8
num_unrollings = 12

class BiGramBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings, bi_grams_list):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    self._bi_grams_list = bi_grams_list
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, bi_grams_vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, bi_gram2id(self._bi_grams_list, self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 2) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities, bi_grams_list):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2bi_gram(bi_grams_list, c) for c in np.argmax(probabilities, 1)]

def batches2string(batches, bi_grams_list):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b, bi_grams_list))]
  return s

train_batches = BiGramBatchGenerator(train_text, batch_size, num_unrollings, bi_grams_list)
valid_batches = BiGramBatchGenerator(valid_text, 1, 1, bi_grams_list)

print('train 1:', batches2string(train_batches.next()), bi_grams_list)
print('train 2:', batches2string(train_batches.next()), bi_grams_list)
print('valid 1:', batches2string(valid_batches.next()), bi_grams_list)
print('valid 2:', batches2string(valid_batches.next()), bi_grams_list)

TypeError: batches2string() missing 1 required positional argument: 'bi_grams_list'

In [124]:
valid_batches.next()[1].shape

(1, 728)

728

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---